In [1]:
from PIL import Image, ImageDraw

import toyplot.font
import toyplot.text

import IPython.display

import xml.etree.ElementTree as xml

In [2]:
text = "foo <span>bar</span> baz"
style = {"font-family":"helvetica", "font-size":16}
fonts = toyplot.font.ReportlabLibrary()

layout = toyplot.text.layout(text=text, style=style, fonts=fonts)
toyplot.text.dump(layout)

<toyplot.text.BlockBox {'font-size': 16.0, 'font-family': 'helvetica'} (0,0) 80.048x14.8>
  <toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (0,0) 26.688x14.8 'foo '>
  <toyplot.text.InlineBox {'font-size': 16.0, 'font-family': 'helvetica'} (26.688,0) 23.12x14.8>
    <toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (26.688,0) 23.12x14.8 'bar'>
  <toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (49.808,0) 30.24x14.8 ' baz'>


In [3]:
def draw(box, image):
    if isinstance(box, toyplot.text.TextBox):
        print box
        xml.SubElement(
            image,
            "rect",
            x=str(box.left),
            y=str(box.top),
            width=str(box.content_width),
            height=str(box.content_height),
            fill="none",
            stroke="black",
        )
        xml.SubElement(
            image,
            "text",
            x=str(box.left),
            y=str(box.bottom),
            fill="black",
            stroke="none",
        ).text = box.text
        
    for child in box.children:
        draw(child, image)

image = xml.Element(
    "svg",
    width=str(layout.content_width+1),
    height=str(layout.content_height+1),
    style="background: #f0f0f0",
)
draw(layout, image)
IPython.display.SVG(xml.tostring(image))

<toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (0,0) 26.688x14.8 'foo '>
<toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (26.688,0) 23.12x14.8 'bar'>
<toyplot.text.TextBox {'font-size': 16.0, 'font-family': 'helvetica'} (49.808,0) 30.24x14.8 ' baz'>
